In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer

# Data base import

In [ ]:
from utils import sample_data

# -- Get the data -- #
NB_SAMPLES = 1000
TRAIN_SAMPLE = Path("../data/samples/sample_{}_train.csv".format(NB_SAMPLES))
VALID_SAMPLE = Path("../data/samples/sample_{}_validation.csv".format(NB_SAMPLES))
TRAIN_SAMPLE = pd.read_csv(TRAIN_SAMPLE).to_numpy()
VALID_SAMPLE = pd.read_csv(VALID_SAMPLE).to_numpy()


# -- Clean the data -- #
from utils.clean_data import clean_data
TRAIN_SAMPLE = clean_data(TRAIN_SAMPLE)
VALID_SAMPLE = clean_data(VALID_SAMPLE)

In [ ]:
print(TRAIN_SAMPLE)

# Pre-processing
#### TWEET_ORIGINALS : List of the tweets : Array, shape = (len(nb_tweets))
#### TWEET_STRINGS : List of the list of the word of each tweet : List of list of string
#### TWEET_SCALARS : List of the description of each tweet : Array, shape = (len(nb_tweets), sentence_size * word_size)
#### IMPORTANT_WORDS : List of the label of each tweet : Array, shape = (len(nb_tweets), sentence_size)

In [ ]:
# -- Parameters -- #
WORD_SIZE = 50  # 50 or 100 or 200 or 300
FILL_WITH = 0  # If a word is not in the dictionary, [0, ..., 0] will describe it.
SENTIMENT_WEIGHT = 2  # Multiply the sentiment by a factor
SENTENCE_SIZE = 50  # What ever
OPTIONS = [WORD_SIZE, SENTENCE_SIZE, FILL_WITH, SENTIMENT_WEIGHT]


# -- Get the original tweets -- #
TWEET_ORIGINALS_TRAIN = TRAIN_SAMPLE[:, 1]
TWEET_ORIGINALS_VALID = VALID_SAMPLE[:, 1]
print("First tweet :")
print(TWEET_ORIGINALS_TRAIN[0])
print("Shape of TWEET_ORIGINAL :", TWEET_ORIGINALS_TRAIN.shape)

In [ ]:
from descriptors.tweet_string.create_strings import create_strings
from descriptors.tokenizer.tokenizer import Tokenizer

# Initialize the tokenizer
TOKENIZER = Tokenizer()

# -- Get the decomposition of the tweets -- #
TWEET_STRINGS_TRAIN = create_strings(TWEET_ORIGINALS_TRAIN, TOKENIZER, SENTENCE_SIZE)
TWEET_STRINGS_VALID = create_strings(TWEET_ORIGINALS_VALID, TOKENIZER, SENTENCE_SIZE)
print("Decomposition of the first tweet :")
print(TWEET_STRINGS_TRAIN[0])
print("Length of TWEET_STRING :", len(TWEET_STRINGS_TRAIN))


In [ ]:
from descriptors.descriptor_glove.descriptor_glove import tweet_scalar_glove
from utils.standardize import standardize


# Get the dictionary
PATH_DICTIONARY = Path("../data/glove_descriptor/glove.6B.{}d.txt".format(WORD_SIZE))
# PATH_DICTIONARY = Path("../data/glove_descriptor/sample_test.txt")
DICTIONARY = pd.read_csv(PATH_DICTIONARY, sep=" ", header=None)

# Additional dictionary
ADDITIONAL_DIC = {"..": "...", "<3": "love"}

# Get the sentiments
SENTIMENTS_TRAIN = TRAIN_SAMPLE[:, -1]
SENTIMENTS_VALID = VALID_SAMPLE[:, -1]

# -- Get the decriptions of each tweets -- #
TWEET_SCALARS_TRAIN = tweet_scalar_glove(TWEET_STRINGS_TRAIN, SENTIMENTS_TRAIN, DICTIONARY, ADDITIONAL_DIC, OPTIONS)
TWEET_SCALARS_VALID = tweet_scalar_glove(TWEET_STRINGS_VALID, SENTIMENTS_VALID, DICTIONARY, ADDITIONAL_DIC, OPTIONS)

# Standardize the tweet descriptions
standardize(TWEET_SCALARS_TRAIN)
standardize(TWEET_SCALARS_VALID)

print("Description of the first tweet :")
print(TWEET_SCALARS_TRAIN[0])
print("Shape of TWEET_SCLALAR :", TWEET_SCALARS_TRAIN.shape)
print(TWEET_SCALARS_VALID.shape)

In [ ]:
from descriptors.tweet_label.create_labels import create_labels

# Create the decompositions of the labels
LABEL_ORIGINALS_TRAIN = TRAIN_SAMPLE[:, 2]
LABEL_ORIGINALS_VALID = VALID_SAMPLE[:, 2]
LABEL_STRINGS_TRAIN = create_strings(LABEL_ORIGINALS_TRAIN, TOKENIZER, SENTENCE_SIZE)
LABEL_STRINGS_VALID = create_strings(LABEL_ORIGINALS_VALID, TOKENIZER, SENTENCE_SIZE)

# -- Get the labels -- #
IMPORTANT_WORDS_TRAIN = create_labels(TWEET_STRINGS_TRAIN, LABEL_STRINGS_TRAIN, SENTENCE_SIZE)
IMPORTANT_WORDS_VALID = create_labels(TWEET_STRINGS_VALID, LABEL_STRINGS_VALID, SENTENCE_SIZE)

IDX = 5
print(TWEET_ORIGINALS_TRAIN[IDX])
print(LABEL_ORIGINALS_TRAIN[IDX])
print("Labels :")
print(IMPORTANT_WORDS_TRAIN[IDX])
print("Shape of IMPORTANT_WORDS :", IMPORTANT_WORDS_TRAIN.shape)

# Classification

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# Try with one KNN
NB_NEIGHBORS = 1

KNN = KNeighborsRegressor(NB_NEIGHBORS, weights="distance")
KNN.fit(TWEET_SCALARS_TRAIN, IMPORTANT_WORDS_TRAIN)

In [ ]:
# --- Optimize the number of neighbors --- #
# Parameter
NB_NEIGHBORS_MAX = 200
NB_NEIGHBORS_MIN = 80
STEP = 10

# Variables
NB_NEIGHBORS_OPT = 1
JACCARD_ACC_MAX = 0
JACCARD_LIST = []

for nb_neigh in range(NB_NEIGHBORS_MIN, NB_NEIGHBORS_MAX, STEP):
    # Define the knn
    KNN = KNeighborsRegressor(nb_neigh, weights="distance")
    
    # Train the knn
    KNN.fit(TWEET_SCALARS_TRAIN, IMPORTANT_WORDS_TRAIN)
    
    # Scalar predictions
    PRED_SCALAR = KNN.predict(TWEET_SCALARS_VALID)
    
    # Get the string predictions
    PRED_STRING = preds_to_strings(TWEET_ORIGINALS_VALID, TWEET_STRINGS_VALID, PRED_SCALAR)
    
    # Compute the loss
    JACCARD_ACC = mean_jaccard(LABEL_ORIGINALS_VALID, PRED_STRING)
    
    # Updates
    JACCARD_LIST.append(JACCARD_ACC)
    if JACCARD_ACC > JACCARD_ACC_MAX:
        JACCARD_ACC_MAX = JACCARD_ACC
        NB_NEIGHBORS_OPT = nb_neigh    

In [ ]:
import matplotlib.pyplot as plt
# Plot the results
PATH_SAVE = Path("../results")

print("The optimal number of neighbors to take is :", NB_NEIGHBORS_OPT)
print("With this number, the Jaccard accuracy is :", JACCARD_ACC_MAX)

plt.plot(np.arange(NB_NEIGHBORS_MIN, NB_NEIGHBORS_MAX, STEP), JACCARD_LIST, label="Jaccard score")
plt.xlabel("nb neighbors")
plt.legend()
plt.savefig(PATH_SAVE / "glove_knn_train_{}_neighbors_{}.jpg".format(NB_SAMPLES, NB_NEIGHBORS_MAX))
plt.show()

In [ ]:
PRED_SCALAR = KNN.predict(TWEET_SCALARS_VALID)
print("Nomber of correct match", np.sum(PREDICTIONS == IMPORTANT_WORDS_VALID))
print("Number of match to make", len(PREDICTIONS) * len(PREDICTIONS[0]))

In [ ]:
from utils.post_processing import preds_to_strings

PRED_STRING = preds_to_strings(TWEET_ORIGINALS_VALID, TWEET_STRINGS_VALID, PRED_SCALAR)

In [ ]:
SHOW_RESULT = False
if SHOW_RESULT:
    for idx_tweet in range(len(RESULTS)):
        print(TWEET_ORIGINALS_VALID[idx_tweet])
        print(IMPORTANT_WORDS_VALID[idx_tweet])
        print(PRED_SCALAR[idx_tweet])
        print(PRED_STRING[idx_tweet], "\n")

In [ ]:
from utils.loss import mean_jaccard

JACCARD_ACC = mean_jaccard(LABEL_ORIGINALS_VALID, PRED_STRING)
print(JACCARD_ACC)